# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

In [1]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 1.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 9.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.6 MB/s eta 0:00:00a 0:00:01
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/opt/venv/lib/python3.9/site-packages/pydantic_core-2.18.4.dist-info'
Check the permissions.


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [2]:
OPEN_AI_API_KEY = 'OPEN AI KEY'

In [5]:
SOURCE_URL = 'https://www.microsoft.com/investor/reports/ar23/' 
HTML_FILEPATH = './html_annnualreport.html'
CSV_FILEPATH_WITH_EMBEDDINGS = './wikipedia_with_embeddings.csv'

In [3]:
# OpenAI Models
EMBEDDING_MODEL = 'text-embedding-3-small'
COMPLETION_MODEL = 'gpt-3.5-turbo'

# Batch size for processing
BATCH_SIZE = 25

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [3]:
# Importing libraries
import requests
import pandas as pd
import openai
from bs4 import BeautifulSoup
from typing import List, Union, Dict
from scipy.spatial.distance import cosine

In [4]:
from openai import OpenAI

ImportError: cannot import name 'OpenAI' from 'openai' (/opt/venv/lib/python3.9/site-packages/openai/__init__.py)

In [8]:
# Function to retrieve HTML page from URL
def retrieve_page(url: str) -> bytes:
    """
    Fetches HTML content from a given URL.
    
    Args:
        url (str): The URL of the webpage to fetch.
        
    Returns:
        bytes: The HTML content of the webpage.
        
    Raises:
        Exception: If there is a connection error.
    """
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers = headers)
    
    if response.status_code == 200:
        return response.content
    else:
        raise Exception('Connection error')

# Save the web page to a file
with open(HTML_FILEPATH, mode='wb') as file:
    page = retrieve_page(SOURCE_URL)
    file.write(page)

In [9]:
print(HTML_FILEPATH)
with open(HTML_FILEPATH) as hf:
    soup = BeautifulSoup(hf, 'html.parser')

print(soup.find_all('title'))

./html_annnualreport.html
[<title>Microsoft 2023 Annual Report</title>]


In [10]:
sections = soup.find_all('article')
df = pd.DataFrame(columns=['section', 'text'])

In [11]:
df.head()

,section,text


In [12]:
for sec in sections:
    if sec.get('id') != None:
        df.loc[len(df.index)]=[sec.get('id'), sec.get_text(separator=' ', strip = True)]
    else:
        df.loc[len(df.index)]=['general-info', sec.get_text(separator=' ', strip = True)]


In [13]:
df.shape

(42, 2)

In [14]:
# Function to extract data from saved HTML
def extract_data(html_file: str) -> pd.DataFrame:
    """
    Extracts data from HTML file.
    
    Args:
        html_file (str): The file path to the HTML file.
        
    Returns:
        pd.DataFrame: A DataFrame containing the extracted data.
    """
    df = pd.DataFrame(columns=['section', 'text'])
    
    with open(html_file) as hf:
        soup = BeautifulSoup(hf, 'html.parser')

    # retrieve all sections within the annual report
    sections = soup.find_all('article')
    
    # determine text within each section and add to dataframe
    for sec in sections:
        if sec.get('id') != None:
            df.loc[len(df.index)]=[sec.get('id'), sec.get_text(separator=' ', strip = True)]
        else:
            df.loc[len(df.index)]=['general-info', sec.get_text(separator=' ', strip = True)]

    return df

In [15]:
# Extracting data from HTML and displaying DataFrame
df = extract_data(HTML_FILEPATH)

In [16]:
print(df.head())
print(df.shape)

                   section                                               text
0             general-info  This next generation of AI will reshape every ...
1             general-info  Financial Review ISSUER PURCHASES OF EQUITY SE...
2     business-description  Business Note About Forward-Looking Statements...
3                  general  General Embracing Our Future Microsoft is a te...
4  human-capital-resources  HUMAN CAPITAL RESOURCES Overview Microsoft aim...
(42, 2)


In [18]:
openai.api_key = OPENAI_API_KEY
openai_client = OpenAI(api_key=OPENAI_API_KEY)

NameError: name 'OpenAI' is not defined

In [1]:
# Function to get embeddings from OpenAI API
def get_embeddings(prompt: Union[str, List[str]], embedding_model: str) -> List[List[float]]:
    """
    Retrieves embeddings from OpenAI API for the given prompt.

    Args:
        prompt (Union[str, List[str]]): Input prompt or list of prompts.
        embedding_model (str): Name of the embedding model to use.

    Returns:
        List[List[float]]: List of embeddings for the input prompt(s).
    """
    response = openai_client.embeddings.create(
        input=prompt if isinstance(prompt, list) else [prompt],
        model=embedding_model
    )
    return [row.embedding for row in response.data]


NameError: name 'Union' is not defined

In [ ]:
# Function to create embeddings for DataFrame
def create_embeddings(df: pd.DataFrame, embedding_model_name: str = EMBEDDING_MODEL, batch_size: int = BATCH_SIZE) -> List[List[float]]:
    """
    Creates embeddings for the text data in the DataFrame.

    Args:
        df (pd.DataFrame): DataFrame containing text data.
        embedding_model_name (str): Name of the embedding model to use.
        batch_size (int): Size of batches for processing.

    Returns:
        List[List[float]]: List of embeddings corresponding to the text data.
    """
    embeddings_output = []
    for index in range(0, len(df), batch_size):
        batch = df.iloc[index:index+batch_size]['text'].tolist()
        embeddings = get_embeddings(batch, embedding_model_name)
        embeddings_output.extend(embeddings)
    return embeddings_output

In [ ]:
# Add embeddings to DataFrame and save to CSV
df['embedding'] = create_embeddings(df)
df.to_csv(CSV_FILEPATH_WITH_EMBEDDINGS, sep=',', index=False)

# Display DataFrame head
print(df.head())

## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [ ]:
def build_simple_prompt(question: str) -> List[Dict[str, str]]:
    """
    Builds a simple prompt for asking a question.

    Args:
        question (str): The question to include in the prompt.

    Returns:
        List[Dict[str, str]]: A list containing a single message with the user role and the provided question.
    """
    return [
        {
            'role': 'user',
            'content': question
        }
    ]

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

### Question 2